In [21]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.stats import skew, kurtosis
import os
import random
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import find_peaks
import pandas as pd
from scipy.fftpack import fft, fftfreq
from scipy.signal import peak_widths, peak_prominences
from scipy.io import loadmat
import seaborn as sns
from scipy.signal import spectrogram

## Dataframe of all signals

##### According to the documents on Z24 only R1V, R2L, R2T, R2V, R3V are present in every setup. Other sensors are moved around during every setup making them difficult to use as reliable channels

In [22]:
def get_avt_files(root):
    folder_path = root
    file_list = os.listdir(folder_path)
    avt_files = []

    # walk inside data folder
    for filename in file_list:
        full_path = os.path.join(folder_path, filename)
        cur_label = int(filename)

        # walk inside data/[number]
        file_nest_list = os.listdir(full_path)
        for filename_1 in file_nest_list:
            if filename_1 != "avt":
                continue
            else:
                # full_path_2 is of the form C:\Users\amroa\Documents\thesis\data\01\avt
                full_path_2 = os.path.join(full_path, filename_1)
                setup_filenames = [filename for filename in os.listdir(full_path_2) if "setup" in filename]
                for avt_file in setup_filenames:
                    full_path_3 = os.path.join(full_path_2, avt_file) # full_path_3 is of the form C:\Users\amroa\Documents\thesis\data\01\avt\01setup09.mat
                    avt = loadmat(full_path_3)
                    avt_files.append(avt)
    return avt_files

In [38]:
def get_dataframes(avt_files):
    """
    This method returns a list of dataframes # type: ignore
    A dataframe is of the form: # type: ignore
    R1V R2L ... R3V  (column names)
    0.1 0.3 ... 0.12
    0.4 0.1 ... 0.22 
    """
    good_sensors = ['R1V ', 'R2L ', 'R2T ', 'R2V ', 'R3V ']
    result = pd.DataFrame(columns=good_sensors)
    list_df = [] # should contain 17 entries, one for each damage state 

    # get pandas dataframes
    for idx, avt in enumerate(avt_files):
        # get index of the good sensor/positions
        arr_sensors = avt['labelshulp']
        bool_array = np.isin(arr_sensors, good_sensors) #  will contain True if entry corresponds to a good sensor
        indices_good_sensors = np.where(bool_array)[0]

        # Create a Pandas DataFrame
        df = pd.DataFrame(data=avt['data'][:, indices_good_sensors], columns=arr_sensors[indices_good_sensors])
        result = pd.concat([result, df], ignore_index=True)

        if (idx+1) % 9 == 0:
            list_df.append(result)
            result = pd.DataFrame(columns=good_sensors)

    return list_df

In [39]:
avt_files = get_avt_files("C:\\Users\\amroa\\Documents\\thesis\\data")
dfs = get_dataframes(avt_files)

In [43]:
for i in range(len(dfs)):
    print(dfs[i].shape)

(589770, 5)
(589770, 5)
(589314, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589698, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)
(589824, 5)


In [3]:
pd.concat((pd.DataFrame(avt['data'],columns = avt[ 'labelshulp']),
pd.DataFrame(avt3['data'],columns = avt3[ 'labelshulp'])))[[ 'R1V ', 'R2L ','R2T ', 'R2V ', 'R3V ']]

NameError: name 'avt' is not defined

In [4]:
avt = loadmat('./data/01/avt/01setup01.mat')
avt3 = loadmat('./data/01/avt/01setup03.mat')

In [20]:
avt3['data'].shape

(65530, 33)

In [10]:
import pandas as pd

# List of dictionaries
dict_list = [avt, avt3]

# Combine data arrays
combined_data = []
for d in dict_list:
    combined_data.extend(d['data'])

# Get labels for columns (assuming labels are consistent across dictionaries)
columns = avt['labelshulp']

# Create DataFrame
df = pd.DataFrame(combined_data, columns=columns)

# Display the DataFrame
df


,99V,100V,101V,102V,103V,199L,199T,199V,200T,200V,...,511T,511V,512L,512T,512V,R1V,R2L,R2T,R2V,R3V
0,-2.866200e-05,-3.647891e-05,-1.148410e-05,-0.000006,0.001453,0.000016,-0.000042,-0.000023,-0.000019,-0.000032,...,0.000011,-0.001053,0.000423,0.004274,0.002850,-0.000024,0.000025,0.000000,4.313776e-05,0.000026
1,-5.790303e-07,-5.790303e-06,9.071475e-06,0.000006,0.001456,0.000018,-0.000041,0.000013,-0.000014,-0.000003,...,0.000013,-0.001047,0.000422,0.004276,0.002862,-0.000006,0.000021,-0.000003,5.935061e-05,0.000027
2,1.032604e-05,-1.930101e-07,1.196663e-05,0.000006,0.001455,0.000018,-0.000022,0.000046,0.000002,0.000009,...,0.000016,-0.001055,0.000411,0.004277,0.002841,0.000023,0.000019,-0.000013,7.961667e-05,-0.000016
3,-6.948364e-06,-2.393325e-05,-7.527394e-06,-0.000012,0.001439,0.000023,-0.000003,0.000021,0.000015,-0.000003,...,0.000005,-0.001048,0.000419,0.004267,0.002845,0.000031,0.000019,-0.000024,9.920720e-05,-0.000037
4,-1.582683e-05,-2.760045e-05,2.895152e-07,0.000004,0.001461,0.000012,0.000004,0.000005,0.000017,-0.000020,...,0.000015,-0.001036,0.000422,0.004279,0.002871,0.000031,0.000021,-0.000027,6.050867e-05,-0.000015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131055,-1.192802e-04,4.840088e-05,-3.474182e-06,0.000012,0.001451,0.000031,-0.000068,-0.000063,0.000011,-0.000076,...,-0.000051,-0.005368,0.000799,0.011140,-0.000006,0.000030,0.000020,0.000002,-1.930101e-07,0.000030
131056,-1.158061e-04,5.151367e-05,-2.702142e-06,0.000010,0.001447,0.000030,-0.000069,-0.000062,0.000013,-0.000074,...,-0.000050,-0.005368,0.000799,0.011144,-0.000007,0.000036,0.000020,0.000000,7.720404e-07,0.000035
131057,-1.212103e-04,4.644775e-05,-4.825253e-06,0.000015,0.001457,0.000031,-0.000062,-0.000062,0.000018,-0.000075,...,-0.000049,-0.005369,0.000801,0.011141,-0.000006,0.000035,0.000020,-0.000003,2.702142e-06,0.000046
131058,-1.223684e-04,4.803467e-05,-1.544081e-06,0.000018,0.001462,0.000029,-0.000066,-0.000064,0.000015,-0.000076,...,-0.000047,-0.005370,0.000803,0.011145,-0.000009,0.000025,0.000021,-0.000003,-1.061556e-05,0.000052
